# Simulating the Atomic Bomb

## Packages

In [ ]:
#using Pkg
#pkg"add SimJulia"
#pkg"add Distributions"
#pkg"add StatsPlots"

using SimJulia
using Distributions
using Plots
using StatsPlots
using CSV
using Logging

## Constants

In [ ]:
const Nₐ = 6.02214086e23  # atoms / mole
const ρᵤ = 19.1           # g / cm3
const mᵤ = 235.0439299    # g / mole
const nᵤ = ρᵤ * Nₐ / mᵤ   # atoms / cm3
const mₙ = 1.008664916    # g / mole
const Mₙ = mₙ / Nₐ * 1e-3 # kg
const k = 1.38064852e-23  # J / K
const q = 1.60217662e-19  # C
const A = mᵤ / mₙ
const α = (A - 1)^2 / (A + 1) ^2
const numberofspontaneousfis = 0.0003; # / g / s
ρᵤ * 4/3 * π * 9^3 * numberofspontaneousfis

## Distributions

In [ ]:
const cosΘdistr = Uniform(-1, 1)
const cosϕdistr = Uniform(-1, 1)

const energy = 1e-3:1e-3:15
function wattspectrum(energy) # MeV
    0.453 * sinh(sqrt(2.29*energy))*exp(-1.036*energy)
end
const spectrum = wattspectrum.(energy)
const wattdistr = Categorical(spectrum ./ sum(spectrum))

const numberofneutronsdistr = Categorical([0,0.6,0.36,0.04])
const numberofneutronsspontaneousdistr = Categorical([0.2,0.74,0.06]);

## Data

In [ ]:
σt = CSV.read("sigma_total.txt")
σf = CSV.read("sigma_fission.txt")
σa = CSV.read("sigma_absorption.txt")
σi = CSV.read("sigma_inelastic.txt")

function Σ(energy::Float64) # 1 / cm
    i = findfirst(e -> e > energy, σt[:, 1])
    σ = σt[i, 2] + (energy - σt[i, 1]) / (σt[i-1, 1] - σt[i, 1]) * (σt[i-1, 2] - σt[i, 2])
    nᵤ * σ * 1e-24
end

function ΔtΔl(energy::Float64)
    Δl = -log(rand()) / Σ(energy)
    v = sqrt(2 * energy * q / Mₙ) * 100
    Δl / v, Δl
end;

## Types

In [ ]:
struct Bomb
    radius :: Float64             # cm
    generated :: Vector{Int64}
    neutrons :: Vector{Int64}
    times :: Vector{Float64}      # s
    function Bomb(radius::Real)
        new(radius, Float64[], Int64[], Float64[])
    end
end

mutable struct Neutron
    r :: Float64                  # cm
    cosθ :: Float64
    energy :: Float64             # eV
    function Neutron(r::Float64, energy::Float64, cosθ::Float64 = rand(cosΘdistr))
        new(r, cosθ, energy)
    end
end

function Neutron(sim::Simulation, bomb::Bomb, r::Float64, energy::Float64=energy[rand(wattdistr)] * 1e6)
    neutron = Neutron(r, energy)
    time = now(sim)
    @info("$time: create neutron at position $r with cosθ = $(neutron.cosθ) and energy = $(neutron.energy) eV")
    push!(bomb.times, time)
    push!(bomb.neutrons, 1)
    Δt, Δl = ΔtΔl(neutron.energy)
    @callback collision(timeout(sim, Δt), bomb, neutron, Δl)
end;

## Callback

In [ ]:
function spontaneousfission(ev::AbstractEvent, bomb::Bomb)
    sim = environment(ev)
    for _ in rand(numberofneutronsspontaneousdistr)
        Neutron(sim, bomb, rand() * bomb.radius)
    end
    rate = ρᵤ * 4/3 * π * bomb.radius^3 * numberofspontaneousfis
    @callback spontaneousfission(timeout(sim, -log(rand()) / rate), bomb)
end

function collision(ev::AbstractEvent, bomb::Bomb, neutron::Neutron, Δl::Float64)
    sim = environment(ev)
    time = now(ev)
    r′ = sqrt(neutron.r^2 + Δl^2 + 2*neutron.r*Δl*neutron.cosθ)
    if r′ > bomb.radius
        @info("$(now(sim)): neutron has left the bomb")
        push!(bomb.times, time)
        push!(bomb.neutrons, -1)
        push!(bomb.generated, 0)
    else
        i = findfirst(e -> e > neutron.energy, σt[:, 1])
        σtot = σt[i, 2] + (neutron.energy - σt[i, 1]) / (σt[i-1, 1] - σt[i, 1]) * (σt[i-1, 2] - σt[i, 2])
        i = findfirst(e -> e > neutron.energy, σf[:, 1])
        σfis = σf[i, 2] + (neutron.energy - σf[i, 1]) / (σf[i-1, 1] - σf[i, 1]) * (σf[i-1, 2] - σf[i, 2])
        i = findfirst(e -> e > neutron.energy, σa[:, 1])
        σabs = σa[i, 2] + (neutron.energy - σa[i, 1]) / (σa[i-1, 1] - σa[i, 1]) * (σa[i-1, 2] - σa[i, 2])
        i = findfirst(e -> e > neutron.energy, σi[:, 1])
        i = i == 1 ? 2 : i
        σin = σi[i, 2] + (neutron.energy - σi[i, 1]) / (σi[i-1, 1] - σi[i, 1]) * (σi[i-1, 2] - σi[i, 2])
        rnd = rand()
        if rnd < σfis / σtot
            n = rand(numberofneutronsdistr)
            @info("$(now(sim)): fission with creation of $n neutrons")
            for _ in 1:n
                Neutron(sim, bomb, r′)
            end
            push!(bomb.times, time)
            push!(bomb.neutrons, -1)
            push!(bomb.generated, n)
        elseif rnd < (σabs + σfis) / σtot
            @info("$(now(sim)): neutron absorbed")
            push!(bomb.times, time)
            push!(bomb.neutrons, -1)
            push!(bomb.generated, 0)
        elseif rnd < (σin + σabs + σfis) / σtot
            @info("$(now(sim)): inelastic scattering")
            n = 1
            Neutron(sim, bomb, r′)
            push!(bomb.times, time)
            push!(bomb.neutrons, -1)
        else
            cosϕ = rand(cosϕdistr)
            cosψ = (A * cosϕ + 1) / sqrt(A^2 + 2 * A * cosϕ +1)
            neutron.r = r′
            neutron.energy *= 0.5 * (1 + α + (1 - α) * cosϕ)
            θ = acos(neutron.cosθ)
            ψ = acos(cosψ)
            θplusψ = θ + ψ
            θminψ = ψ < π / 2 ? θ - ψ : θ - ψ + 2π
            neutron.cosθ = cos(θplusψ + rand() * (θminψ - θplusψ))
            @info("$(now(sim)): elastic scattering at position $r′ with cosθ = $(neutron.cosθ) and energy = $(neutron.energy) eV")
            Δt, Δl = ΔtΔl(neutron.energy)
            @callback collision(timeout(sim, Δt), bomb, neutron, Δl)
        end
    end
    ((sum(bomb.generated) > 500 && sum(bomb.neutrons) == 0) || (time > 1 && sum(bomb.neutrons) == 0) || sum(bomb.generated) > 1000) && throw(StopSimulation())
end

## Simulation

In [ ]:
sim = Simulation()
bomb = Bomb(8.0)
@callback spontaneousfission(timeout(sim, 0.0), bomb)
run(sim)

In [ ]:
mean(bomb.generated)

## Plot

In [ ]:
i = findlast(x->x==0, cumsum(bomb.neutrons))
i = i === nothing ? 1 : i
plot(bomb.times[i+1:end], cumsum(bomb.neutrons)[i+1:end], seriestype=:scatter, ylabel="N", xlabel="time [s]")
#plot(bomb.times, cumsum(bomb.neutrons), seriestype=:scatter, ylabel="N", xlabel="time [s]")

## Monte Carlo

In [ ]:
const RUNS = 100
const RADII = 5:12;
Logging.disable_logging(LogLevel(1000));

In [ ]:
ks = zeros(Float64, RUNS, length(RADII))
for (i, r) in enumerate(RADII)
    for j in 1:RUNS
        sim = Simulation()
        bomb = Bomb(r)
        @callback spontaneousfission(timeout(sim, 0.0), bomb)
        run(sim)
        ks[j, i] = mean(bomb.generated)
    end
end

In [ ]:
boxplot(reshape(collect(RADII), 1, length(RADII)), ks, label=reshape(collect(RADII), 1, length(RADII)), legend=:bottomright, xlabel="R [cm]", ylabel="k")

In [ ]:
mean(ks, dims=1)

In [ ]:
plot(RADII, [mean(ks, dims=1) ...], seriestype=:scatter, xlabel="R [cm]", ylabel="k")